In [10]:
import pandas as pd
import MetaTrader5 as mt
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import FunctionTransformer
from tsfresh.feature_extraction import EfficientFCParameters
from tsfresh import extract_features, select_features
from tsfresh.utilities.dataframe_functions import impute
import joblib

from methods.make_single_df_from_bid_ask import make_single_df_from_bid_ask
from methods.make_dataframe_line import make_dataframe_line
# from methods.make_5min_volume_from_1_min_volume import make_5min_volume_from_1min_volume
from machine_learning_models.lstm.eurusd.lstm_predict_candle import predict_candle
# from machine_learning_models.lstm.eurusd.lstm_use_prediction import use_prediction
from machine_learning_models.lstm.lstm_collect_predictions_into_dataframe import collect_predictions_into_dataframe
from methods.make_dataframe_from_server_for_training import make_dataframe_from_server_for_training
# import torch

import matplotlib.pyplot as plt

import seaborn as sns

In [25]:
COLUMNS_ORDER = [
        "open",
        "high",
        "low",
        "close",
        "volume",
        "ADX",
        "ADL",
        "ATR_14",
        "RSI",
        "MACD",
        "MACD_signal",
        "MACD_hist",
        "open_audusd",
        "high_audusd",
        "low_audusd",
        "close_audusd",
        "volume_audusd",
        "open_brentusd",
        "high_brentusd",
        "low_brentusd",
        "close_brentusd",
        "volume_brentusd",
        "open_cadusd",
        "high_cadusd",
        "low_cadusd",
        "close_cadusd",
        "volume_cadusd",
        "open_jpyusd",
        "high_jpyusd",
        "low_jpyusd",
        "close_jpyusd",
        "volume_jpyusd"
    ]

In [12]:
columns_for_y = [
        "open_plus_5min",
        "high_plus_5min",
        "low_plus_5min",
        "close_plus_5min",
        "open_plus_10min",    
        "high_plus_10min",
        "low_plus_10min",
        "close_plus_10min",
        "open_plus_15min",
        "high_plus_15min",
        "low_plus_15min",
        "close_plus_15min",
        "open_plus_20min",
        "high_plus_20min",
        "low_plus_20min",
        "close_plus_20min",
        "open_plus_25min",
        "high_plus_25min",
        "low_plus_25min",
        "close_plus_25min",
        "open_plus_30min",
        "high_plus_30min",
        "low_plus_30min",
        "close_plus_30min",
        "open_plus_35min",
        "high_plus_35min",
        "low_plus_35min",
        "close_plus_35min",
        "open_plus_40min",
        "high_plus_40min",
        "low_plus_40min",
        "close_plus_40min"
        ]

In [13]:
if not mt.initialize():
    print("initialize() failed, error code =", mt.last_error())
    quit()

account=52159692
pw = 'D!1a8p2ON$mwQR'

authorized = mt.login(account, pw)

In [14]:
# Display all rows
pd.set_option('display.max_rows', None)

# Display all columns
pd.set_option('display.max_columns', None)

# Prevent column width truncation
pd.set_option('display.max_colwidth', None)

# Display full content without wrapping
pd.set_option('display.expand_frame_repr', False)

In [15]:
df_test = make_dataframe_from_server_for_training(timeframe=mt.TIMEFRAME_M5, start_pos=0, end_pos=300, columns_order=COLUMNS_ORDER)

In [16]:
df_test.to_csv('dataframe_for_training.csv')

In [17]:
df_test.head()

,open,high,low,close,volume,ADX,ADL,ATR_14,RSI,MACD,MACD_signal,MACD_hist,open_audusd,high_audusd,low_audusd,close_audusd,volume_audusd,open_brentusd,high_brentusd,low_brentusd,close_brentusd,volume_brentusd,open_cadusd,high_cadusd,low_cadusd,close_cadusd,volume_cadusd,open_jpyusd,high_jpyusd,low_jpyusd,close_jpyusd,volume_jpyusd,open_plus_5min,high_plus_5min,low_plus_5min,close_plus_5min,open_plus_10min,high_plus_10min,low_plus_10min,close_plus_10min,open_plus_15min,high_plus_15min,low_plus_15min,close_plus_15min,open_plus_20min,high_plus_20min,low_plus_20min,close_plus_20min,open_plus_25min,high_plus_25min,low_plus_25min,close_plus_25min,open_plus_30min,high_plus_30min,low_plus_30min,close_plus_30min,open_plus_35min,high_plus_35min,low_plus_35min,close_plus_35min,open_plus_40min,high_plus_40min,low_plus_40min,close_plus_40min
time,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
2025-06-13 03:35:00,1.15636,1.15645,1.15477,1.15481,843,51.072761,-2324.569542,0.001129,34.327601,-0.001123,-0.000289,-0.000834,0.64864,0.64874,0.64685,0.64692,695,72.35,72.43,72.05,72.18,420,1.36168,1.36279,1.36162,1.36279,454,142.945,142.952,142.793,142.853,925,1.15483,1.15509,1.15396,1.15461,1.15459,1.15595,1.15425,1.15559,1.15559,1.15626,1.15485,1.15625,1.15625,1.15625,1.15505,1.15590,1.15590,1.15602,1.15424,1.15425,1.15423,1.15546,1.15423,1.15510,1.15510,1.15511,1.15413,1.15466,1.15465,1.15485,1.15396,1.15407
2025-06-13 03:40:00,1.15483,1.15509,1.15396,1.15461,670,48.780423,-2223.773082,0.001129,33.613531,-0.001378,-0.000507,-0.000871,0.64696,0.64759,0.64696,0.64733,580,72.18,72.38,72.10,72.31,390,1.36278,1.36345,1.36248,1.36327,407,142.853,143.045,142.853,142.965,773,1.15459,1.15595,1.15425,1.15559,1.15559,1.15626,1.15485,1.15625,1.15625,1.15625,1.15505,1.15590,1.15590,1.15602,1.15424,1.15425,1.15423,1.15546,1.15423,1.15510,1.15510,1.15511,1.15413,1.15466,1.15465,1.15485,1.15396,1.15407,1.15407,1.15452,1.15390,1.15428
2025-06-13 03:45:00,1.15459,1.15595,1.15425,1.15559,610,45.439461,-1872.126023,0.001177,40.256643,-0.001182,-0.000642,-0.000540,0.64731,0.64826,0.64728,0.64795,621,72.31,72.89,72.30,72.88,456,1.36330,1.36345,1.36228,1.36252,478,142.971,143.190,142.921,143.152,761,1.15559,1.15626,1.15485,1.15625,1.15625,1.15625,1.15505,1.15590,1.15590,1.15602,1.15424,1.15425,1.15423,1.15546,1.15423,1.15510,1.15510,1.15511,1.15413,1.15466,1.15465,1.15485,1.15396,1.15407,1.15407,1.15452,1.15390,1.15428,1.15428,1.15500,1.15400,1.15499
2025-06-13 03:50:00,1.15559,1.15626,1.15485,1.15625,627,43.510812,-1254.019640,0.001196,44.348104,-0.000847,-0.000683,-0.000164,0.64795,0.64800,0.64743,0.64770,555,72.87,73.80,72.79,73.72,750,1.36252,1.36319,1.36247,1.36296,553,143.156,143.175,142.923,142.991,892,1.15625,1.15625,1.15505,1.15590,1.15590,1.15602,1.15424,1.15425,1.15423,1.15546,1.15423,1.15510,1.15510,1.15511,1.15413,1.15466,1.15465,1.15485,1.15396,1.15407,1.15407,1.15452,1.15390,1.15428,1.15428,1.15500,1.15400,1.15499,1.15499,1.15516,1.15450,1.15460
2025-06-13 03:55:00,1.15625,1.15625,1.15505,1.15590,537,41.740928,-1030.269640,0.001196,42.658043,-0.000711,-0.000688,-0.000022,0.64770,0.64802,0.64724,0.64772,576,73.72,74.37,73.47,73.56,986,1.36293,1.36332,1.36259,1.36271,463,142.987,143.089,142.965,143.043,747,1.15590,1.15602,1.15424,1.15425,1.15423,1.15546,1.15423,1.15510,1.15510,1.15511,1.15413,1.15466,1.15465,1.15485,1.15396,1.15407,1.15407,1.15452,1.15390,1.15428,1.15428,1.15500,1.15400,1.15499,1.15499,1.15516,1.15450,1.15460,1.15460,1.15462,1.15373,1.15425


In [27]:
def generate_automatic_features_for_model_training(df_raw:pd.DataFrame, cols_order:list, column_for_y:str) -> tuple[pd.DataFrame, list]:
    columns_order = cols_order.copy()
    df_data = df_raw.loc[:, columns_order]
    df_data[column_for_y] = df_test[column_for_y]
    df_data.reset_index(inplace=True)
    df_data.dropna(axis=1, inplace=True)
    df_data['id'] = df_data.index
    columns_order.append('id')
    columns_order.append('time')

    extracted_features = extract_features(
        df_data.loc[:, columns_order],
        column_id="id",  
        column_sort="time", 
        default_fc_parameters=EfficientFCParameters(),          
        impute_function=impute, 
    )

    y = df_data[column_for_y] 

    features_filtered = select_features(extracted_features, y)

    feature_columns = features_filtered.columns.to_list()

    print(f"Original number of features: {len(extracted_features.columns)}")
    print(f"Number of features after selection: {len(features_filtered.columns)}")
    print("Some selected features:")
    print(features_filtered.head())

    return features_filtered, feature_columns

In [ ]:
features_filtered, feature_columns = generate_automatic_features_for_model_training(df_raw=df_test, cols_order=COLUMNS_ORDER, column_for_y='open_plus_40min')

Feature Extraction: 100%|██████████| 30/30 [01:12<00:00,  2.40s/it]


Original number of features: 24864
Number of features after selection: 583
Some selected features:
   close__maximum  close__absolute_maximum  close__quantile__q_0.1  close__minimum  close__mean  close__root_mean_square  close__cwt_coefficients__coeff_0__w_10__widths_(2, 5, 10, 20)  close__cwt_coefficients__coeff_0__w_5__widths_(2, 5, 10, 20)  close__cwt_coefficients__coeff_0__w_2__widths_(2, 5, 10, 20)  close__quantile__q_0.9  close__quantile__q_0.8  close__quantile__q_0.7  close__quantile__q_0.6  close__quantile__q_0.4  close__quantile__q_0.3  close__quantile__q_0.2  close__abs_energy  close__median  close__sum_values  close__cwt_coefficients__coeff_0__w_20__widths_(2, 5, 10, 20)  close__fft_coefficient__attr_"real"__coeff_0  close__fft_coefficient__attr_"abs"__coeff_0  low__quantile__q_0.7  low__quantile__q_0.4  low__cwt_coefficients__coeff_0__w_2__widths_(2, 5, 10, 20)  low__cwt_coefficients__coeff_0__w_5__widths_(2, 5, 10, 20)  low__cwt_coefficients__coeff_0__w_20__widths_(2, 5, 1

In [33]:
def select_columns(X, columns=feature_columns):
    return X[columns]

def generate_automatic_features_for_model_test(df_raw:pd.DataFrame, cols_order:list, feature_columns:list) -> pd.DataFrame:
    columns_order = cols_order.copy()
    df_data = df_raw.loc[:, columns_order]
    df_data.reset_index(inplace=True)
    df_data.dropna(axis=1, inplace=True)
    df_data['id'] = df_data.index

    if('id' not in columns_order):
        columns_order.append('id')

    if('time' not in columns_order):
        columns_order.append('time')

    extracted_features = extract_features(
        df_data.loc[:, columns_order],
        column_id="id",  
        column_sort="time", 
        default_fc_parameters=EfficientFCParameters(),          
        impute_function=impute, 
    )

    pipeline = Pipeline([
    ('imputer', FunctionTransformer(impute)),
    ('selector', FunctionTransformer(select_columns))
    ])

    production_ready_features = pipeline.transform(extracted_features)

    print(f"Original number of features: {len(extracted_features.columns)}")
    print(f"Number of features after selection: {len(production_ready_features.columns)}")
    print("Some selected features:")
    print(production_ready_features.head())

    return production_ready_features.loc[:, feature_columns]

In [ ]:
production_ready_features = generate_automatic_features_for_model_test(df_raw=df_test, cols_order=COLUMNS_ORDER)

Feature Extraction: 100%|██████████| 30/30 [01:14<00:00,  2.49s/it]


Original number of features: 24864
Number of features after selection: 583
Some selected features:
   close__maximum  close__absolute_maximum  close__quantile__q_0.1  close__minimum  close__mean  close__root_mean_square  close__cwt_coefficients__coeff_0__w_10__widths_(2, 5, 10, 20)  close__cwt_coefficients__coeff_0__w_5__widths_(2, 5, 10, 20)  close__cwt_coefficients__coeff_0__w_2__widths_(2, 5, 10, 20)  close__quantile__q_0.9  close__quantile__q_0.8  close__quantile__q_0.7  close__quantile__q_0.6  close__quantile__q_0.4  close__quantile__q_0.3  close__quantile__q_0.2  close__abs_energy  close__median  close__sum_values  close__cwt_coefficients__coeff_0__w_20__widths_(2, 5, 10, 20)  close__fft_coefficient__attr_"real"__coeff_0  close__fft_coefficient__attr_"abs"__coeff_0  low__quantile__q_0.7  low__quantile__q_0.4  low__cwt_coefficients__coeff_0__w_2__widths_(2, 5, 10, 20)  low__cwt_coefficients__coeff_0__w_5__widths_(2, 5, 10, 20)  low__cwt_coefficients__coeff_0__w_20__widths_(2, 5, 1

In [29]:
production_ready_features.head()

,close__maximum,close__absolute_maximum,close__quantile__q_0.1,close__minimum,close__mean,close__root_mean_square,"close__cwt_coefficients__coeff_0__w_10__widths_(2, 5, 10, 20)","close__cwt_coefficients__coeff_0__w_5__widths_(2, 5, 10, 20)","close__cwt_coefficients__coeff_0__w_2__widths_(2, 5, 10, 20)",close__quantile__q_0.9,close__quantile__q_0.8,close__quantile__q_0.7,close__quantile__q_0.6,close__quantile__q_0.4,close__quantile__q_0.3,close__quantile__q_0.2,close__abs_energy,close__median,close__sum_values,"close__cwt_coefficients__coeff_0__w_20__widths_(2, 5, 10, 20)","close__fft_coefficient__attr_""real""__coeff_0","close__fft_coefficient__attr_""abs""__coeff_0",low__quantile__q_0.7,low__quantile__q_0.4,"low__cwt_coefficients__coeff_0__w_2__widths_(2, 5, 10, 20)","low__cwt_coefficients__coeff_0__w_5__widths_(2, 5, 10, 20)","low__cwt_coefficients__coeff_0__w_20__widths_(2, 5, 10, 20)","low__cwt_coefficients__coeff_0__w_10__widths_(2, 5, 10, 20)",low__quantile__q_0.9,low__quantile__q_0.6,low__quantile__q_0.2,low__quantile__q_0.3,low__minimum,low__absolute_maximum,low__maximum,low__root_mean_square,low__mean,low__median,low__abs_energy,low__quantile__q_0.1,low__sum_values,low__quantile__q_0.8,"low__fft_coefficient__attr_""real""__coeff_0","low__fft_coefficient__attr_""abs""__coeff_0","high__fft_coefficient__attr_""real""__coeff_0",high__quantile__q_0.9,"high__cwt_coefficients__coeff_0__w_5__widths_(2, 5, 10, 20)","high__cwt_coefficients__coeff_0__w_20__widths_(2, 5, 10, 20)",high__mean,high__median,high__abs_energy,high__sum_values,high__minimum,high__absolute_maximum,high__maximum,high__quantile__q_0.1,high__quantile__q_0.6,high__quantile__q_0.7,high__quantile__q_0.8,high__root_mean_square,high__quantile__q_0.4,"high__cwt_coefficients__coeff_0__w_10__widths_(2, 5, 10, 20)","high__fft_coefficient__attr_""abs""__coeff_0","high__cwt_coefficients__coeff_0__w_2__widths_(2, 5, 10, 20)",high__quantile__q_0.2,high__quantile__q_0.3,open__quantile__q_0.7,"open__cwt_coefficients__coeff_0__w_10__widths_(2, 5, 10, 20)","open__cwt_coefficients__coeff_0__w_5__widths_(2, 5, 10, 20)",open__quantile__q_0.6,open__quantile__q_0.4,open__quantile__q_0.8,open__maximum,"open__fft_coefficient__attr_""real""__coeff_0","open__cwt_coefficients__coeff_0__w_20__widths_(2, 5, 10, 20)",open__abs_energy,open__root_mean_square,"open__cwt_coefficients__coeff_0__w_2__widths_(2, 5, 10, 20)",open__quantile__q_0.9,open__mean,open__quantile__q_0.2,open__quantile__q_0.1,open__minimum,open__quantile__q_0.3,open__absolute_maximum,open__median,open__sum_values,"open__fft_coefficient__attr_""abs""__coeff_0",ADL__mean,ADL__quantile__q_0.2,ADL__quantile__q_0.3,ADL__quantile__q_0.4,ADL__quantile__q_0.6,ADL__maximum,ADL__minimum,ADL__sum_values,ADL__quantile__q_0.1,"ADL__cwt_coefficients__coeff_0__w_10__widths_(2, 5, 10, 20)","ADL__cwt_coefficients__coeff_0__w_5__widths_(2, 5, 10, 20)","ADL__fft_coefficient__attr_""real""__coeff_0","ADL__cwt_coefficients__coeff_0__w_2__widths_(2, 5, 10, 20)",ADL__quantile__q_0.7,ADL__quantile__q_0.8,ADL__quantile__q_0.9,"ADL__cwt_coefficients__coeff_0__w_20__widths_(2, 5, 10, 20)",ADL__median,"ADL__fft_coefficient__attr_""angle""__coeff_0",ADL__count_below__t_0,ADL__range_count__max_1000000000000.0__min_0,ADL__range_count__max_0__min_-1000000000000.0,ADL__count_above__t_0,"high_audusd__fft_coefficient__attr_""abs""__coeff_0",high_audusd__quantile__q_0.4,high_audusd__absolute_maximum,high_audusd__median,"high_audusd__cwt_coefficients__coeff_0__w_20__widths_(2, 5, 10, 20)",high_audusd__quantile__q_0.2,high_audusd__quantile__q_0.1,high_audusd__minimum,high_audusd__quantile__q_0.3,"high_audusd__fft_coefficient__attr_""real""__coeff_0",high_audusd__quantile__q_0.8,"high_audusd__cwt_coefficients__coeff_0__w_2__widths_(2, 5, 10, 20)","high_audusd__cwt_coefficients__coeff_0__w_5__widths_(2, 5, 10, 20)","high_audusd__cwt_coefficients__coeff_0__w_10__widths_(2, 5, 10, 20)",high_audusd__quantile__q_0.9,high_audusd__quantile__q_0.6,high_audusd__quant

In [32]:
features_filtered.loc[:, feature_columns].head()

,close__maximum,close__absolute_maximum,close__quantile__q_0.1,close__minimum,close__mean,close__root_mean_square,"close__cwt_coefficients__coeff_0__w_10__widths_(2, 5, 10, 20)","close__cwt_coefficients__coeff_0__w_5__widths_(2, 5, 10, 20)","close__cwt_coefficients__coeff_0__w_2__widths_(2, 5, 10, 20)",close__quantile__q_0.9,close__quantile__q_0.8,close__quantile__q_0.7,close__quantile__q_0.6,close__quantile__q_0.4,close__quantile__q_0.3,close__quantile__q_0.2,close__abs_energy,close__median,close__sum_values,"close__cwt_coefficients__coeff_0__w_20__widths_(2, 5, 10, 20)","close__fft_coefficient__attr_""real""__coeff_0","close__fft_coefficient__attr_""abs""__coeff_0",low__quantile__q_0.7,low__quantile__q_0.4,"low__cwt_coefficients__coeff_0__w_2__widths_(2, 5, 10, 20)","low__cwt_coefficients__coeff_0__w_5__widths_(2, 5, 10, 20)","low__cwt_coefficients__coeff_0__w_20__widths_(2, 5, 10, 20)","low__cwt_coefficients__coeff_0__w_10__widths_(2, 5, 10, 20)",low__quantile__q_0.9,low__quantile__q_0.6,low__quantile__q_0.2,low__quantile__q_0.3,low__minimum,low__absolute_maximum,low__maximum,low__root_mean_square,low__mean,low__median,low__abs_energy,low__quantile__q_0.1,low__sum_values,low__quantile__q_0.8,"low__fft_coefficient__attr_""real""__coeff_0","low__fft_coefficient__attr_""abs""__coeff_0","high__fft_coefficient__attr_""real""__coeff_0",high__quantile__q_0.9,"high__cwt_coefficients__coeff_0__w_5__widths_(2, 5, 10, 20)","high__cwt_coefficients__coeff_0__w_20__widths_(2, 5, 10, 20)",high__mean,high__median,high__abs_energy,high__sum_values,high__minimum,high__absolute_maximum,high__maximum,high__quantile__q_0.1,high__quantile__q_0.6,high__quantile__q_0.7,high__quantile__q_0.8,high__root_mean_square,high__quantile__q_0.4,"high__cwt_coefficients__coeff_0__w_10__widths_(2, 5, 10, 20)","high__fft_coefficient__attr_""abs""__coeff_0","high__cwt_coefficients__coeff_0__w_2__widths_(2, 5, 10, 20)",high__quantile__q_0.2,high__quantile__q_0.3,open__quantile__q_0.7,"open__cwt_coefficients__coeff_0__w_10__widths_(2, 5, 10, 20)","open__cwt_coefficients__coeff_0__w_5__widths_(2, 5, 10, 20)",open__quantile__q_0.6,open__quantile__q_0.4,open__quantile__q_0.8,open__maximum,"open__fft_coefficient__attr_""real""__coeff_0","open__cwt_coefficients__coeff_0__w_20__widths_(2, 5, 10, 20)",open__abs_energy,open__root_mean_square,"open__cwt_coefficients__coeff_0__w_2__widths_(2, 5, 10, 20)",open__quantile__q_0.9,open__mean,open__quantile__q_0.2,open__quantile__q_0.1,open__minimum,open__quantile__q_0.3,open__absolute_maximum,open__median,open__sum_values,"open__fft_coefficient__attr_""abs""__coeff_0",ADL__mean,ADL__quantile__q_0.2,ADL__quantile__q_0.3,ADL__quantile__q_0.4,ADL__quantile__q_0.6,ADL__maximum,ADL__minimum,ADL__sum_values,ADL__quantile__q_0.1,"ADL__cwt_coefficients__coeff_0__w_10__widths_(2, 5, 10, 20)","ADL__cwt_coefficients__coeff_0__w_5__widths_(2, 5, 10, 20)","ADL__fft_coefficient__attr_""real""__coeff_0","ADL__cwt_coefficients__coeff_0__w_2__widths_(2, 5, 10, 20)",ADL__quantile__q_0.7,ADL__quantile__q_0.8,ADL__quantile__q_0.9,"ADL__cwt_coefficients__coeff_0__w_20__widths_(2, 5, 10, 20)",ADL__median,"ADL__fft_coefficient__attr_""angle""__coeff_0",ADL__count_below__t_0,ADL__range_count__max_1000000000000.0__min_0,ADL__range_count__max_0__min_-1000000000000.0,ADL__count_above__t_0,"high_audusd__fft_coefficient__attr_""abs""__coeff_0",high_audusd__quantile__q_0.4,high_audusd__absolute_maximum,high_audusd__median,"high_audusd__cwt_coefficients__coeff_0__w_20__widths_(2, 5, 10, 20)",high_audusd__quantile__q_0.2,high_audusd__quantile__q_0.1,high_audusd__minimum,high_audusd__quantile__q_0.3,"high_audusd__fft_coefficient__attr_""real""__coeff_0",high_audusd__quantile__q_0.8,"high_audusd__cwt_coefficients__coeff_0__w_2__widths_(2, 5, 10, 20)","high_audusd__cwt_coefficients__coeff_0__w_5__widths_(2, 5, 10, 20)","high_audusd__cwt_coefficients__coeff_0__w_10__widths_(2, 5, 10, 20)",high_audusd__quantile__q_0.9,high_audusd__quantile__q_0.6,high_audusd__quant

In [ ]:
df_combined = pd.merge(df_data, features_filtered, how="inner", left_index=True, right_index=True)

In [ ]:
df_combined.head()

In [ ]:
len(df_combined.columns)

In [ ]:


# Create processing function

# 

In [ ]:
production_ready_features.head()

In [ ]:
df_data.head()

In [ ]:
from pandas.testing import assert_frame_equal


In [ ]:
test_empty_dataframe = pd.DataFrame()

In [ ]:
assert_frame_equal(production_ready_features, features_filtered, check_exact=True)

In [ ]:
fig = plt.figure(figsize=(10, 15))

In [ ]:
symbols = [
    'AUDUSD',
    'XBRUSD',
    'USDJPY',
    'CADUSD'
]

In [ ]:
TIMEFRAME_SMALL_MT = mt.TIMEFRAME_M5

In [ ]:
df_for_corr = make_dataframe_line(timeframe=TIMEFRAME_SMALL_MT, start_pos=0, end_pos=500)

In [ ]:
df_for_corr = df_for_corr.drop(['+DI', '-DI', 'ADX', 'ADL', 'ATR_14', 'RSI', 'MACD', 'MACD_signal', 'MACD_hist'], axis=1)

In [ ]:
corr = df_for_corr.corr()

In [ ]:
corr

In [ ]:
sns.set_theme(rc={'figure.figsize':(20,14)})

In [ ]:
sns.heatmap(corr,
            xticklabels=corr.columns.values,
            yticklabels=corr.columns.values,
            annot=True)

In [ ]:
df_from_server = make_dataframe_line(timeframe=TIMEFRAME_SMALL_MT, start_pos=0, end_pos=92)

In [ ]:
len(df_from_server)

In [ ]:
BASE_DIR = 'hist_data/'
TIME_SERIES_FOLDER = 'EURUSD/'
BID_FOLDER = '5_min/Bid/'
ASK_FOLDER = '5_min/Ask/'

BASE_DIR_LSTM = 'machine_learning_models/lstm/eurusd/pickle_files'
LSTM_MAX_VAL = BASE_DIR_LSTM + '/lstm_regressor_predict_max_values-2025-04-07.pkl'
LSTM_MIN_VAL = BASE_DIR_LSTM + '/lstm_regressor_predict_min_values-2025-04-07.pkl'
SCALER_X_MIN = BASE_DIR_LSTM + '/lstm_regressor_scaler_x_min.pkl'
SCALER_X_MAX = BASE_DIR_LSTM + '/lstm_regressor_scaler_x_max.pkl'
SCALER_Y_MIN = BASE_DIR_LSTM + '/lstm_regressor_scaler_y_min.pkl'
SCALER_Y_MAX = BASE_DIR_LSTM + '/lstm_regressor_scaler_y_max.pkl'

In [ ]:
len(df_from_server)

In [ ]:
df_joined_train, df_joined_test, df_joined_val = make_single_df_from_bid_ask(
    base_dir=BASE_DIR,
    time_series_folder=TIME_SERIES_FOLDER
)

In [ ]:
print(len(df_joined_train), len(df_joined_test), len(df_joined_val))

In [ ]:
columns_for_y = [
        "open_plus_5min",
        "high_plus_5min",
        "low_plus_5min",
        "close_plus_5min",
        "open_plus_10min",    
        "high_plus_10min",
        "low_plus_10min",
        "close_plus_10min",
        "open_plus_15min",
        "high_plus_15min",
        "low_plus_15min",
        "close_plus_15min",
        "open_plus_20min",
        "high_plus_20min",
        "low_plus_20min",
        "close_plus_20min",
        "open_plus_25min",
        "high_plus_25min",
        "low_plus_25min",
        "close_plus_25min",
        "open_plus_30min",
        "high_plus_30min",
        "low_plus_30min",
        "close_plus_30min",
        "open_plus_35min",
        "high_plus_35min",
        "low_plus_35min",
        "close_plus_35min",
        "open_plus_40min",
        "high_plus_40min",
        "low_plus_40min",
        "close_plus_40min"
    ]

In [ ]:
for col in columns_for_y:
    predict_candle(df=df_joined_train, test_df=df_joined_test, validation_df=df_joined_val, base_dir=BASE_DIR_LSTM, column_for_y=col)

In [ ]:
dataframe_actual = df_joined_val.tail(8)

In [ ]:
len(df_joined_val)

In [ ]:
dataframe_line = df_joined_val.iloc[4547:4608]

In [ ]:
len(dataframe_line)

In [ ]:
dataframe_predicted = collect_predictions_into_dataframe(dataframe_line=dataframe_line, base_dir_lstm=BASE_DIR_LSTM)

In [ ]:
dataframe_predicted_from_server = collect_predictions_into_dataframe(dataframe_line=df_from_server, base_dir_lstm=BASE_DIR_LSTM)

In [ ]:
dataframe_predicted_from_server

In [ ]:
df_test = dataframe_actual[['open', 'high', 'low', 'close']].copy()

In [ ]:
df_test.index = dataframe_predicted.index.to_list()

In [ ]:
df_test

In [ ]:
dataframe_predicted

In [ ]:
df_result = df_test.subtract(dataframe_predicted, axis=1)

In [ ]:
df_result.mean()

In [ ]:
dataframe_predicted.add(0.041)

In [ ]:
# predict_min_stats(df_joined_train, df_joined_test, df_joined_val, scaler_x_dir=SCALER_X_MIN, scaler_y_dir=SCALER_Y_MIN, model_dir=LSTM_MIN_VAL)

In [ ]:
# predict_max_stats(df_joined_train, df_joined_test, df_joined_val, scaler_x_dir=SCALER_X_MAX, scaler_y_dir=SCALER_Y_MAX, model_dir=LSTM_MAX_VAL)